In [14]:
using ProtoSyn
using Printf

## **1. data**

1. load dihedrals definitions
2. load state

In [22]:
dihd_json = Aux.read_JSON("data/1ctf_mc_top.json")
dihedrals, residues = Mutators.Dihedral.load_topology(dihd_json);

state = Common.load_from_pdb("data/mol.pdb")
state.energy = Common.Energy();

## **2. function definitions**

### **2.1 sampler**

In [23]:
dihedral_mutator = Mutators.Dihedral.DihedralMutator(
    dihedrals   # list of dihedrals
    , 0.1       # single dihedral mutation probability
    , randn     # angle sampler
    , 1.0       # stepsize
)
my_sampler(st::Common.State) = Mutators.Dihedral.run!(st, dihedral_mutator);

### **2.2 evaluator**

In [24]:
function my_evaluator(st::Common.State, do_forces::Bool)
    st.energy.eTotal
end;

### **2.3 MC driver**

In [25]:
mc_driver = Drivers.MonteCarlo.MonteCarloDriver(
    my_sampler      # sampler
    , my_evaluator  # evaluator
    , 1.0           # temperature
    , 1000          # nsteps
);

### **2.4 callback**

This function is used to:
1. control the driver's output
2. update the mutator's stepsize

In [1]:
function callback(step::Int64, st::Common.State, dr::Drivers.MonteCarlo.MonteCarloDriver, ac_ratio::Float64)
    write(stdout, @sprintf "(MC) Step: %4d | Energy: %9.4f\n" step state.energy.eTotal)
    dihedral_mutator.stepsize *= 0.95
end

my_callback = Common.CallbackObject(
    100          # calling frequency
    , callback  # the actual callback function
);



cb = @callback 10 function(step::Int64, st::Common.State, dr::Drivers.MonteCarlo.MonteCarloDriver, naccept::Int64)
    # do something
    println("do something")
end

LoadError: UndefVarError: @sprintf not defined

## **3. run**

In [32]:
Drivers.MonteCarlo.run!(state, mc_driver, my_callback)

(MC) Step:  100 | Energy:    0.0000
(MC) Step:  200 | Energy:    0.0000
(MC) Step:  300 | Energy:    0.0000
(MC) Step:  400 | Energy:    0.0000
(MC) Step:  500 | Energy:    0.0000
(MC) Step:  600 | Energy:    0.0000
(MC) Step:  700 | Energy:    0.0000
(MC) Step:  800 | Energy:    0.0000
(MC) Step:  900 | Energy:    0.0000
(MC) Step: 1000 | Energy:    0.0000


In [2]:
using PyPlot

ArgumentError: ArgumentError: Package PyPlot not found in current path:
- Run `Pkg.add("PyPlot")` to install the PyPlot package.
